In [ ]:
from queue import Queue
import os

def leer_grafo(path):
    with open(path, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]
    n = int(lines[0])
    cap = [[0]*n for _ in range(n)]
    for line in lines[1:]:
        u, v, c = map(int, line.split())
        cap[u][v] = c
    return n, cap

def bfs_camino_aumentante(R, source, sink):
    # BFS para encontrar camino aumentante y reconstruir path desde sink a source
    n = len(R)
    visitado = [0]*n
    label = [0]*n
    predecesor = [-1]*n

    q = Queue()
    q.put(source)
    visitado[source] = 1
    label[source] = float('inf')

    while not q.empty():
        u = q.get()
        for j in range(n):
            if R[u][j] > 0 and visitado[j] == 0:
                label[j] = R[u][j]
                visitado[j] = 1
                predecesor[j] = u
                q.put(j)
    
    if visitado[sink] == 0:
        return None, 0, predecesor
    
    i = sink
    path, flujo = [], []
    while True:
        path.append(i)
        flujo.append(label[i])
        if i == source:
            break
        i = predecesor[i]
    
    flujo.reverse()
    path.reverse()
    cuello = min(flujo[1:n]) if len(flujo) > 1 else 0
    return path, cuello, predecesor

def edmonds_karp(n, cap, source, sink):
    # Algoritmo de Edmonds-Karp para encontrar flujo máximo
    R = [row[:] for row in cap]
    max_flow = 0
    augmenting_paths = []
    while True:
        path, cuello, _ = bfs_camino_aumentante(R, source, sink)
        if not path:
            break
        max_flow += cuello
        augmenting_paths.append((path, cuello))
        for i in range(len(path)-1):
            u, v = path[i], path[i+1]
            R[u][v] -= cuello
            R[v][u] += cuello
    return max_flow, augmenting_paths
base = os.path.join(os.path.dirname(__file__), 'grafos') if '__file__' in globals() else 'grafos'
archivos = [
    'flow-grafo.txt',
    'flow-grafo-2.txt',
    'flow-grafo-3.txt',
    'flow-grafo-4.txt',
    'flow-grafo-5.txt',
]

resultados = []
for nombre in archivos:
    path = os.path.join(base, nombre)
    n, cap = leer_grafo(path)
    source, sink = 0, n-1
    max_flow, aug_paths = edmonds_karp(n, cap, source, sink)
    resultados.append((nombre, max_flow, aug_paths))

for nombre, max_flow, aug_paths in resultados:
    print('Archivo:', nombre)
    print('Máximo flujo:', max_flow)
    for k, (p, f) in enumerate(aug_paths, 1):
        print('  Camino', k, ':', p, '+', f)
    print('-'*50)